In [1]:
import ast
import json
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

/Users/cjlovering/workspace/venv/torch/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
def load_jsonl(path, key1="sent1_str", key2="sent2_str"):
    out = []

    def strip(x):
        return "-".join(x.split()).lower()

    def get_key(row):
        return strip(row[key1]) + strip(row[key2])

    with open(path, "r") as f:
        for l in f.readlines():
            out.append(json.loads(l))

    df = pd.DataFrame(out)
    df["key"] = df.apply(get_key, axis=1)
    return df


def load_df(path, key1="sent1_str", key2="sent2_str"):
    out = []

    def strip(x):
        return "-".join(x).lower()

    def get_key(row):
        return strip(ast.literal_eval(row[key1])) + strip(ast.literal_eval(row[key2]))
    
    df = pd.read_csv(path, sep="\t")
    df["key"] = df.apply(get_key, axis=1)
    df = df.rename(
        {"0": "sent1", "1": "sent2", "2": "label"}
    )
    return df


In [5]:
df_tokenized = load_df(f"../data/NEP/all-tokenized.tsv")
df_random_tokenized = load_df(f"../data/NEP-random/all-tokenized.tsv")

PATH = "13-april-2020"
df = load_jsonl(f"{PATH}/nep_test.jsonl")
df_random = load_jsonl(f"{PATH}/nep-random_test.jsonl")

df_merged = df_tokenized.merge(df, on="key")
df_random_merged = df_random_tokenized.merge(df_random, on="key")

In [7]:
temp = df_random_merged[['sent_1',
 'sent_2',
 'label',
  'labels',
 'preds',
 'positive_1',
 'positive_2',
 'subsequence_1',
 'subsequence_2',
]]
temp.to_csv(f"{PATH}/random-results.tsv", sep='\t', index=False)

In [8]:
temp = df_merged[['sent_1',
 'sent_2',
 'label',
   'labels',
 'preds',
 'case',
 'cue',
 'common_cue',
]]
temp.to_csv(f"{PATH}/results.tsv", sep='\t', index=False)

### Cases

In [16]:
# original
temp = df_merged.copy()
temp["correct"] = temp.labels == temp.preds
temp.groupby(["label", "case"])[
    ["correct"]].mean()

correct
label         case                      
contradiction a: within scope.  0.002060
              b: cue-removed    0.012685
entailment    c: a S clause     0.981447

### Original Exp.

In [18]:
temp = df_merged.copy()
temp = temp[temp.case == "a: within scope."]
temp["correct"] = 1. * (temp.labels == temp.preds)
temp["pred_neg"] = 1. * (temp.preds == 2)
temp["pred_pos"] = 1. * (temp.preds == 1)

temp.groupby(["cue"])[
    ["correct"]].agg(['count','mean'])

correct          
                  count      mean
cue                              
-less                12  0.000000
-lessly              10  0.100000
Not-not               2  0.000000
ab-                  13  0.000000
by-no-means           2  0.000000
dis-                  5  0.000000
fail                  9  0.000000
far-from              1  0.000000
im-                  33  0.000000
in-                  23  0.000000
ir-                   9  0.000000
neglected             1  0.000000
neither-nor          13  0.000000
neither-nor-nor       1  0.000000
never               130  0.000000
never-more            1  0.000000
no                  257  0.000000
no-longer            13  0.076923
no-more               3  0.000000
no-nor                2  0.000000
nobody               12  0.000000
non-                  8  0.000000
none                 10  0.000000
nor                   5  0.000000
not                 739  0.000000
not-nor               1  0.000000
not-nor-nor           1  0.000000
nothing              78  0.000000
nowhere               2  0.000000
prevent               1  0.000000
rather-than           1  0.000000
save                  2  0.000000
un-                  51  0.019608
without               5  0.000000

### ``but''

In [10]:
temp["but_1"] = temp.sent_1.apply(lambda x: "but" in x)
temp["but_2"] = temp.sent_2.apply(lambda x: "but" in x)

temp.groupby(["label", "but_1", "but_2", "case"])[["correct"]].agg(['count','mean'])

correct          
                                             count      mean
label         but_1 but_2 case                              
contradiction False False a: within scope.    1221  0.001638
                          b: cue-removed      1543  0.009721
              True  False a: within scope.     229  0.004367
                          b: cue-removed        98  0.040816
                    True  a: within scope.       6  0.000000
                          b: cue-removed       251  0.019920
entailment    False False c: a S clause        711  0.988748
              True  False c: a S clause        340  0.964706
                    True  c: a S clause         27  1.000000

### Random

In [19]:
temp = df_random_merged.copy()
temp["correct"] = temp.labels == temp.preds
temp["pred_neg"] = temp.preds == 2
temp["pred_pos"] = temp.preds == 1

temp.groupby(["positive_1", "positive_2", "subsequence_1", "subsequence_2"])[
    ["correct", "pred_neg", "pred_pos"]].mean()

correct  pred_neg  \
positive_1 positive_2 subsequence_1 subsequence_2                       
False      False      False         False          0.055344  0.758588   
                                    True           0.143145  0.725806   
                      True          False          0.062500  0.891129   
                                    True           0.154605  0.723684   
           True       False         True           0.149243  0.680193   
                      True          True           0.181818  0.676768   
True       False      True          False          0.053645  0.871389   
                                    True           0.148428  0.719497   
           True       True          True           0.189903  0.664995   

                                                   pred_pos  
positive_1 positive_2 subsequence_1 subsequence_2            
False      False      False         False          0.186069  
                                    True           0.131048  
                      True          False          0.046371  
                                    True           0.121711  
           True       False         True           0.170564  
                      True          True           0.141414  
True       False      True          False          0.074966  
                                    True           0.132075  
           True       True          True           0.145102

In [20]:
temp.groupby(["positive_1", "positive_2"])[
    ["correct", "pred_neg", "pred_pos"]].mean()

correct  pred_neg  pred_pos
positive_1 positive_2                              
False      False       0.088311  0.775171  0.136519
           True        0.160730  0.678985  0.160285
True       False       0.087150  0.817697  0.095153
           True        0.189903  0.664995  0.145102

### Length

In [13]:
# original
temp = df_merged.copy()
temp["correct"] = temp.labels == temp.preds
temp["len_premise"] = temp.sent_1.apply(len)
temp["len_hypothesis"] = temp.sent_2.apply(len)

temp.groupby(["correct", "label"])[
    ["len_premise", "len_hypothesis"]].mean()

len_premise  len_hypothesis
correct label                                     
False   contradiction   115.709425       74.402590
        entailment      180.150000       46.900000
True    contradiction   136.407407       63.185185
        entailment      154.085066       51.907372

In [14]:
temp.groupby(["correct", "label", "case"])[
    ["len_premise", "len_hypothesis"]].mean()

len_premise  len_hypothesis
correct label         case                                         
False   contradiction a: within scope.   106.583620       39.569855
                      b: cue-removed     122.807816      101.496788
        entailment    c: a S clause      180.150000       46.900000
True    contradiction a: within scope.   180.666667       32.666667
                      b: cue-removed     130.875000       67.000000
        entailment    c: a S clause      154.085066       51.907372

In [15]:
# random
temp = df_random_merged.copy()
temp["correct"] = temp.labels == temp.preds
temp["pred_neg"] = temp.preds == 2
temp["pred_pos"] = temp.preds == 1
temp["len_premise"] = temp.sent_1.apply(len)
temp["len_hypothesis"] = temp.sent_2.apply(len)

temp.groupby(["positive_1", "positive_2", "correct"])[
    ["len_premise", "len_hypothesis"]].mean()

len_premise  len_hypothesis
positive_1 positive_2 correct                             
False      False      False      85.318203       85.987365
                      True       84.231884       77.323671
           True       False      84.897613       60.395756
                      True       78.869806       57.171745
True       False      False      60.993668       84.565514
                      True       54.239796       74.454082
           True       False      60.264135       60.789930
                      True       55.927817       53.716549